### Chuyển vi-vocab của java thành file text

In [1]:
import codecs
import re

def read_binary_file(filename):
    # Đọc nội dung của tệp bin
    with open(filename, 'rb') as f:
        content = f.read()

    # Giải mã các byte dữ liệu thành chuỗi Unicode
    decoded_content = codecs.decode(content, 'utf-8', 'backslashreplace') # các ký tự đặc biệt thành ?

    # Loại bỏ các ký tự không phải Unicode khỏi chuỗi
    contents = [c if c.isprintable() else '@' for c in decoded_content]
    filtered_content = ''.join(contents)
    ls = re.split(r'(t@)', filtered_content)
    # lọc bỏ các 't@
    result = [x[1:] for x in ls if x != 't@']
    return result[1:]

data = read_binary_file('data/vi-vocab')
# sắp lại theo độ dài (số lượng từ) từ dài đến ngắn, từ A đến Z trong cùng độ dài
data.sort(key=lambda x: (-len(x.split()), x))
# ghi ra file
with open('data/vi-vocab.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write("%s\n" % item)
print(len(data))

34203


### Chuyển từ điển json sang text

In [8]:
import re

data = []
with open('data/Viet74K.txt', 'r') as file:
    for line in file:
        line = line.replace(',', ' , ')
        line = re.sub(r'\s+', ' ', line)
        line = line.lower().strip()
        if '-' in line:
            data.append(line.replace('-', ' '))
        data.append(line)
data= list(set(data))
# sắp lại theo độ dài (số lượng từ) từ dài đến ngắn, từ A đến Z trong cùng độ dài
data.sort(key=lambda x: (-len(re.split(r'\s|\-',x)), x))
# loại bỏ các từ không có dấu cách
data = [x for x in data if len(re.split(r'\s|\-',x)) > 1]

with open('data/dic.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write("%s\n" % item)

print(len(data))

66767


### Tổng hợp từ từ file csv


In [2]:
import os
import re

def getFiles(folder):
    files = []
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            files.append(os.path.join(folder, file))
    return files

def getFilesData(files):
    data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                words = [x for x in line.split(',')]                
                data += words
    return data

data = getFilesData(getFiles('data\\words'))

data= list(set(data))
data = [re.sub(r'[^\w]+', ' ', x).strip().lower() for x in data]
# sắp lại theo độ dài (số lượng từ) từ dài đến ngắn, từ A đến Z trong cùng độ dài
data.sort(key=lambda x: (-len(re.split(r'\s|\-',x)), x))
# loại bỏ các từ không có dấu cách
data = [x for x in data if len(re.split(r'\s|\-',x)) > 1]

with open('data/dic2.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write("%s\n" % item)

print(len(data))

75556


### Tổng hợp

In [3]:
# Kết hợp 2 file dic.txt và dic2.txt
data = []
with open('data/dic.txt', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(line.strip())
with open('data/dic2.txt', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(line.strip())

data= list(set(data))
data.sort(key=lambda x: (-len(re.split(r'\s|\-',x)), x))

with open('data/dic3.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write("%s\n" % item)

print(len(data))

112342


### Tạo Stopword
Code này copy bên file [Text classification.ipynb](Text%20classification.ipynb) qua, nên nếu muốn chạy thì copy qua bên đó , chỗ stopword thì và chạy lại

In [ ]:
from tqdm import tqdm
# Hàm word count
def word_count(lines):
    word_count = defaultdict(int)
    counter = 0
    for i in (pbar := tqdm(range(len(lines)))):
        document = lines[i]
        # chuẩn hóa cách gõ dấu tiếng Việt
        document = vietnameseTextNormalizer(document)
        # tách từ
        document = tokenize_sentences(document)
        # đưa về lower
        document = document.lower()
        # xóa các ký tự không cần thiết
        document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
        # xóa khoảng trắng thừa
        document = re.sub(r'\s+', ' ', document).strip()
        for word in document.split():
            word_count[word] += 1
            counter += 1
         # Bổ sung thông tin vào thanh tiến trình của tqdm
        pbar.set_description(f"words:{counter}")
    return word_count

# Tạo word count cho tập train + test
word_count_result = word_count([x[0] for x in train_data + test_data])
# xoá phần tử không phải văn bản
word_count_OK = {k: v for k, v in word_count_result.items() if k.isnumeric() == False}
# in 100 từ xuất hiện nhiều nhất
sorted(word_count_OK.items(), key=lambda x: x[1], reverse=True)[:200]

In [ ]:
# xoá phần tử không phải văn bản
word_count_OK = {k: v for k, v in word_count_result.items() if k.isnumeric() == False}
# in 100 từ xuất hiện nhiều nhất
sorted(word_count_OK.items(), key=lambda x: x[1], reverse=True)[:200]